# Install the <b>simpletransformers</b> library (restart runtime if using Google Colab)

In [ ]:
! pip install simpletransformers
! pip install tokenizers==0.9.4

# Download the datasets(files converted to xlsx)

In [ ]:
## Tamil
! gdown https://drive.google.com/uc?id=10pPg_WI0Qzgvi-qwxcyWbtfqYda0DsoM
! gdown https://drive.google.com/uc?id=1iF4sZ1XFL4pG6YVGrPWEsN1NMiPDQ0cA
! gdown https://drive.google.com/uc?id=1CgoMCL-ZKda6G8xfrVeNPPiSt2S6hBio


## Malayalam
! gdown https://drive.google.com/uc?id=1aA-cxg_iRtM83NgCDSluBIL7fsNUiJg_
! gdown https://drive.google.com/uc?id=1y50Xnd685oCoziVytpVtBcJ8CZizE5Nt
! gdown https://drive.google.com/uc?id=1h7vrLgccRuEanpDipHMFv2q4ptX65HlW


## Kannada
! gdown https://drive.google.com/uc?id=1k6on-7xMJ6zyaFpCxrV3CZii4y1UD4KP
! gdown https://drive.google.com/uc?id=1Dx-TByQ2gIjvHmmNJTA-Aj8rqwGjHo7i
! gdown https://drive.google.com/uc?id=1zG-K2hdpx4n-Geqpww7s5bd8CYCgeJBH

# Import required modules

In [ ]:
import pandas as pd
import numpy as np
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import f1_score, accuracy_score, classification_report, matthews_corrcoef
from sklearn.metrics import precision_recall_fscore_support as score
import os
import tarfile
import warnings 

warnings.filterwarnings('ignore')

# Choose language 

In [ ]:
lang = input('Choose language: 1 for tamil, 2 for malayalam, 3 for kannada: ')

class_list = ['Not_offensive',
 'Offensive_Targeted_Insult_Group',
 'Offensive_Targeted_Insult_Individual',
 'Offensive_Targeted_Insult_Other',
 'Offensive_Untargetede']


train_file_name = None
dev_file_name = None

if lang == '1': 
    lang = 'Tamil'
    class_list.append('not-Tamil')
    train_file_name = 'tamil_offensive_full_train.xlsx'
    dev_file_name = 'tamil_offensive_full_dev.xlsx'
    test_file_name = 'tamil_offensive_full_test_with_labels.xlsx'
elif lang == '2': 
    lang = 'malayalam'
    # As it contains only 5 classes ('Offensive_Targeted_Insult_Other' is not present)
    class_list.append('not-malayalam')
    class_list.remove('Offensive_Targeted_Insult_Other')

    train_file_name = 'mal_full_offensive_train.xlsx'
    dev_file_name = 'mal_full_offensive_dev.xlsx'
    test_file_name = 'mal_full_offensive_test_with_labels.xlsx'
elif lang == '3': 
    lang = 'Kannada'
    class_list.append('not-Kannada')
    train_file_name = 'kannada_offensive_train.xlsx'
    dev_file_name = 'kannada_offensive_dev.xlsx'(test_df['Input'].apply(lambda val: str(val))
    test_file_name = 'kannada_offensive_test_with_labels.xlsx'

# Load data into dataframes

In [ ]:
train_df = pd.read_excel(train_file_name, header=None)
train_df.columns = ['Input', 'Label']
train_df = train_df.dropna().drop_duplicates().reset_index(drop=True, inplace=False)

dev_df = pd.read_excel(dev_file_name, header=None)
dev_df.columns = ['Input', 'Label']

test_df = pd.read_excel(test_file_name, header=None, engine='openpyxl')
test_df.columns = ['Input', 'Label']

# Labels mapped to integers
train_df['Label'] = train_df.apply(lambda x:  class_list.index(x['Label']),axis=1)
dev_df['Label'] = dev_df.apply(lambda x:  class_list.index(x['Label']),axis=1)
test_df['Label'] = test_df.apply(lambda x:  class_list.index(x['Label']),axis=1)


print(f'Number of exmaples in the train set: {train_df.shape[0]}')
print(f'Number of exmaples in the validation set: {dev_df.shape[0]}')
print(f'Number of exmaples in the test set: {test_df.shape[0]}')

# How sample data looks like.

In [ ]:
test_df.head(10)

# Class-weighting with inverse of #samples in the class

In [ ]:
inverse_weights = np.array(train_df['Label'].value_counts().sort_index())
weights = np.sum(inverse_weights) / inverse_weights

# Load pre-trained model

In [ ]:
# define hyperparameters
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 10, # run for 10 epochs
             "no_save": True, # don't save the weights after each iteration as it exceeds the runtime memory allowed
             }

class_weighting = input('Do you want to use class weighting:\nPress\nY for Yes\nN for No: ')

if class_weighting.lower() == 'n':
    # Create a Classification Model (WITHOUT USING CLASS-WEIGHTING)
    print('Model not using class-weighting')
    model = ClassificationModel(
        "distilbert", "distilbert-base-multilingual-cased",
        num_labels=len(class_list),
        args=train_args
)

else:
    # Create a Classification Model (USING CLASS-WEIGHTING)
    print('Model using class-weighting')
    model = ClassificationModel(
        "distilbert", "distilbert-base-multilingual-cased",
        num_labels=len(class_list),
        weight=list(weights),
        args=train_args
    )

# Train the model

In [ ]:
def to_str(text):
    tt = text['Input']
    if not isinstance(tt, str):
        tt = str(tt)
    return pd.Series([tt, text['Label']])

model.train_model(train_df.apply(lambda row: to_str(row), axis=1))

# Check performance over test set

## Metrics

In [ ]:
# precision_recall_fscore_support
def scr(labels, preds):
    return score(labels, preds, average='macro')

# f1 score
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
# classification report
def classification_rprt(labels, preds):
    return classification_report(labels, preds, output_dict=True, target_names=class_list)

# matthews correlation coefficient 
def mathews_coff(labels, pred):
    return matthews_corrcoef(labels, pred)

## Evaluate over the validation set

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(dev_df.apply(lambda x: to_str(x), axis=1), f1=f1_multiclass, acc=accuracy_score, 
                                                            cls_report=classification_rprt, mathews_coff = mathews_coff, score=scr)

print(f"Val acc: {result['acc']}\nEval_loss: {result['eval_loss']}\nf1 score: {result['f1']}\nMatthews corrcoeff: {result['mathews_coff']}")

## The Classification report

In [ ]:
pd.DataFrame(result['cls_report']).transpose()

## The precision, recall and f1 scores over the dev-set



In [ ]:
print(f"Precision: \t{result['score'][0]} \nRecall: \t{result['score'][1]} \nF1-score: \t{result['score'][2]}")

# Make a dataframe to write the results to an excel file

In [ ]:
if lang == 'Kannada' or lang == 'Tamil':
    result_df = pd.concat([pd.DataFrame({'Input': [test_df.iloc[i]['Input']],
                            'Correct Label': [class_list[test_df.iloc[i]['Label']]], 
                            'Predicted Label': [class_list[np.argmax(model_outputs[i])]],
                            'Score_Not_offensive': [model_outputs[i][0]],
                            'Score_Offensive_Targeted_Insult_Group': [model_outputs[i][1]],
                            'Score_Offensive_Targeted_Insult_Individual': [model_outputs[i][2]],
                            'Score_Offensive_Targeted_Insult_Other': [model_outputs[i][3]],
                            'Score_Offensive_Untargetede': [model_outputs[i][4]],
                            f'Score_not-{lang}': [model_outputs[i][5]]
                            
                            }) for i, j in enumerate(model_outputs)],  ignore_index=True)
    
else:
    result_df = pd.concat([pd.DataFrame({'Input': [test_df.iloc[i]['Input']],
                            'Correct Label': [class_list[test_df.iloc[i]['Label']]], 
                            'Predicted Label': [class_list[np.argmax(model_outputs[i])]],
                            'Score_Not_offensive': [model_outputs[i][0]],
                            'Score_Offensive_Targeted_Insult_Group': [model_outputs[i][1]],
                            'Score_Offensive_Targeted_Insult_Individual': [model_outputs[i][2]],
                            'Score_Offensive_Untargetede': [model_outputs[i][3]],
                            f'Score_not-{lang}': [model_outputs[i][4]]
                            
                            }) for i, j in enumerate(model_outputs)],  ignore_index=True)

## Normalize the scores

In [ ]:
# To normalize the scores between 0 and 1 (both inclusive)
req_cols = [col for col in result_df.columns if col.startswith('Score')]

def normalize(row):
    vals = np.array([row[col] for col in req_cols])
    req_vals = np.exp(vals)/sum(np.exp(vals))
    for i,col in enumerate(req_cols):
        row[col] = req_vals[i]

    return pd.Series(row)

# Write results to file

In [ ]:
result_df = result_df.apply(lambda row: normalize(row), axis=1)
result_df.to_excel(f'mbert_val_{lang}_without_psefind_probsudo-labelling.xlsx', encoding='utf-8')

# Pseudo-labelling

### 1) Obtain predictions over test-set

In [ ]:
test_df['Input'] = test_df['Input'].apply(lambda val: str(val))
predictions, raw_outputs = model.predict(test_df['Input'])

### 2) Combine the test-set inputs and predictions obtained with the train-set

In [ ]:
test_df['Label'] = predictions
pseudo_label_df = pd.concat([train_df, test_df], axis=0).reset_index(drop=True)

### 3) Train the model again with pseudo-labelled data included 

In [ ]:
model.train_model(pseudo_label_df)

# Check performance over dev-set with the new model

### Evaluate over the validation set

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(dev_df.apply(lambda x: to_str(x), axis=1), f1=f1_multiclass, acc=accuracy_score, 
                                                            cls_report=classification_rprt, mathews_coff = mathews_coff, score=scr)

print(f"Val acc: {result['acc']}\nEval_loss: {result['eval_loss']}\nf1 score: {result['f1']}\nMatthews corrcoeff: {result['mathews_coff']}")

## The Classification Report

In [ ]:
pd.DataFrame(result['cls_report']).transpose()

## The precision, recall and f1 scores over the dev-set

In [ ]:
print(f"Precision: {result['score'][0]} \nRecall: {result['score'][1]} \nF1-score: {result['score'][2]}")

# Make a dataframe to write the results to an excel file

In [ ]:
if lang == 'Kannada' or lang == 'Tamil':
    result_df = pd.concat([pd.DataFrame({'Input': [dev_df.iloc[i]['Input']],
                            'Correct Label': [class_list[dev_df.iloc[i]['Label']]], 
                            'Predicted Label': [class_list[np.argmax(model_outputs[i])]],
                            'Score_Not_offensive': [model_outputs[i][0]],
                            'Score_Offensive_Targeted_Insult_Group': [model_outputs[i][1]],
                            'Score_Offensive_Targeted_Insult_Individual': [model_outputs[i][2]],
                            'Score_Offensive_Targeted_Insult_Other': [model_outputs[i][3]],
                            'Score_Offensive_Untargetede': [model_outputs[i][4]],
                            f'Score_not-{lang}': [model_outputs[i][5]]
                            
                            }) for i, j in enumerate(model_outputs)],  ignore_index=True)
    
else:
    result_df = pd.concat([pd.DataFrame({'Input': [dev_df.iloc[i]['Input']],
                            'Correct Label': [class_list[dev_df.iloc[i]['Label']]], 
                            'Predicted Label': [class_list[np.argmax(model_outputs[i])]],
                            'Score_Not_offensive': [model_outputs[i][0]],
                            'Score_Offensive_Targeted_Insult_Group': [model_outputs[i][1]],
                            'Score_Offensive_Targeted_Insult_Individual': [model_outputs[i][2]],
                            'Score_Offensive_Untargetede': [model_outputs[i][3]],
                            f'Score_not-{lang}': [model_outputs[i][4]]
                            
                            }) for i, j in enumerate(model_outputs)],  ignore_index=True)

## Write results to file

In [ ]:
result_df = result_df.apply(lambda row: normalize(row), axis=1)
result_df.to_excel(f'mbert_val_{lang}_with_pseudo-labelling.xlsx', encoding='utf-8')